In [1]:
import json
import os
import requests
import sys
current_directory = os.getcwd()
project_directory = current_directory.split("/")[:-1]
project_directory = "/".join(project_directory)
sys.path.insert(1, project_directory)
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from wbe_odm import odm
from wbe_odm.odm_mappers import mcgill_mapper
from wbe_odm.odm_mappers import excel_template_mapper

In [2]:
def load_from_mcgill(path_to_lab_data, path_to_static_data, path_to_types, path_to_mapping, sheet_name, lab_id, start_date=None, end_date=None):
    mapper = mcgill_mapper.McGillMapper()
    mapper.read(
        path_to_lab_data,
        path_to_static_data,
        path_to_types,
        path_to_mapping,
        sheet_name,
        lab_id,
        startdate=start_date,
        enddate=end_date
    )
    odm_data = odm.Odm()
    odm_data.load_from(mapper)
    return odm_data


def get_available_sites(odm_data):
    samples = odm_data.sample
    available_sites = list(samples["siteID"].unique())
    site_dico = {}
    for i, site in enumerate(available_sites):
        if site == "":
            continue
        site_dico[i] = site
    return site_dico

def apply_quality_flag(odm_data, sample_ids, meas_types):
    """Puts the quality flag to True for the given measurement types"""
    measures = odm_data.ww_measure
    sample_filt = (measures["sampleID"].isin(sample_ids))
    type_filt = (measures["type"].isin(meas_types))
    measures.loc[sample_filt & type_filt, "qualityFlag"] = True
    setattr(odm_data, "ww_measure", measures)
    return odm_data

def get_normalized_data(odm_data, site_id):
    def sample_time(row):
        comp_start = row["dateTimeStart"]
        grab = row["dateTime"]
        if pd.isna(comp_start):
            return grab
        else:
            return comp_start
    samples = odm_data.sample
    samples["reportDate"] = samples.apply(sample_time, axis=1)
    samples_from_site = samples.loc[samples["siteID"] == site_id, "sampleID"].unique()
    def date_from_sample_id(sample_id):
        sample_slice = samples.loc[samples["sampleID"] == sample_id]
        try:
            reportDate = sample_slice.iloc[0]["reportDate"]
            return reportDate
        except IndexError:
            return pd.NaT
    measures = odm_data.ww_measure
    sample_filt = (measures["sampleID"].isin(samples_from_site))
    viral_filt = (measures["type"].isin(["covN2", "nPMMoV"]))
    unit_filt = (measures["unit"] == "gc/ml")
    quality_filt = (measures["qualityFlag"] == False)
    measures = measures.loc[sample_filt & viral_filt & unit_filt & quality_filt, ["sampleID", "type", "value"]]
    measures["sampleDate"] = measures["sampleID"].apply(date_from_sample_id)
    measures = measures.drop_duplicates(subset=["sampleID", "type"])
    measures = measures.pivot(index=["sampleID", "sampleDate"], columns="type", values="value")
    if "covN2" not in measures.columns:
        print(f"no sars measurements found for site {site_id}")
        return None
    measures["normalized"] = measures["covN2"] / measures["nPMMoV"]
    measures = measures.sort_values("sampleDate")
    measures = measures.reset_index()
    return measures.dropna(subset=["normalized"])

### Quebec



In [3]:
path_to_folder = "../Data/Lab/McGill/"
lab_data = "/Users/jeandavidt/Downloads/20210317(b)_Results Template (McGill) Quebec.xlsx"
static_data = "/Users/jeandavidt/Downloads/Ville de Quebec - All data - v1.1.xlsx"
types = path_to_folder + "mcgill_types.csv"
mapping = path_to_folder + "mcgill_map.csv"
sheet_name = "QC Data Daily Samples (McGill)"
lab_id = "modeleau_lab"
start_date = "2021-01-01"
end_date = None
qc_lab_data = load_from_mcgill(lab_data, static_data, types, mapping, sheet_name, lab_id, start_date=start_date, end_date=end_date)


/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [4]:
bad_samples = [
    "qc_1_cptp24h_pstgrit_2021-02-12_1",
    "qc_1_cptp24h_pstgrit_2021-02-13_1",
    "qc_1_cptp24h_pstgrit_2021-02-14_1",
    "qc_1_cptp24h_pstgrit_2021-02-19_1",
    "qc_1_cptp24h_pstgrit_2021-02-20_1",
    "qc_1_cptp24h_pstgrit_2021-02-21_1",
    "qc_1_cptp24h_pstgrit_2021-02-22_1",
    "qc_1_cptp24h_pstgrit_2021-02-23_1",
    "qc_1_cptp24h_pstgrit_2021-03-01_1",
    "qc_1_cptp24h_pstgrit_2021-03-23_1",
    
    "qc_2_cptp24h_pstgrit_2021-02-12_1",
    "qc_2_cptp24h_pstgrit_2021-02-13_1",
    "qc_2_cptp24h_pstgrit_2021-02-14_1",
    "qc_2_cptp24h_pstgrit_2021-02-15_1",
    "qc_2_cptp24h_pstgrit_2021-02-16_1",
    "qc_2_cptp24h_pstgrit_2021-02-19_1",
    "qc_2_cptp24h_pstgrit_2021-02-20_1",
    "qc_2_cptp24h_pstgrit_2021-02-21_1",
    "qc_2_cptp24h_pstgrit_2021-02-22_1",
    "qc_2_cptp24h_pstgrit_2021-02-23_1",
    "qc_2_cptp24h_pstgrit_2021-03-23_1",
    "qc_2_cptp24h_pstgrit_2021-03-24_1",
    
]
qc_lab_data = apply_quality_flag(qc_lab_data, bad_samples, ["covN2", "nPMMoV"])

### Montreal data

In [5]:
path_to_folder = "../Data/Lab/McGill/"
lab_data = "/Users/jeandavidt/Downloads/20210317(b)_Results Template (McGill) Montreal.xlsx"
static_data = path_to_folder + "mcgill_static.xlsx"
types = path_to_folder + "mcgill_types.csv"
mapping = path_to_folder + "mcgill_map.csv"
sheet_name = "Mtl Data Daily Samples (McGill)"
lab_id = "frigon_lab"
start_date = "2021-01-01"
end_date = None
mcgill_lab_data = load_from_mcgill(lab_data, static_data, types, mapping, sheet_name, lab_id, start_date=start_date, end_date=end_date)

/usr/local/Caskroom/miniconda/base/envs/covid/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [6]:
existing_odm = odm.Odm()
excel_mapper = excel_template_mapper.ExcelTemplateMapper()
excel_mapper.read(static_data)
existing_odm.load_from(excel_mapper)
existing_odm.append_from(mcgill_lab_data)
existing_odm.append_from(qc_lab_data)

In [7]:
def get_site_name(odm_data, site_id):
    sites = odm_data.site
    return sites.loc[sites["siteID"] == site_id].iloc[0].loc["name"]

def prettify_name(name):
    name_lst = name.split(" ")
    name_lst = [x.title() for x in name_lst]
    name = " ".join(name_lst)
    name = name.replace("Quebec", "Québec")
    name = name.replace("Montreal", "Montréal")
    return name

def graph_normalized(odm_data, region_name, site_list, max_normalized):
    fig = make_subplots(rows=1, cols=1,
                    specs=[[{"secondary_y": True}]])
    traces = []
    for site in site_list:
        name = get_site_name(odm_data, site)
        site_measures = get_normalized_data(odm_data, site)
        if site_measures is None:
            print(site)
            continue
        trace = go.Scatter(
            x=site_measures["sampleDate"],
            y=site_measures["normalized"],
            name=prettify_name(name),
            mode="lines+markers",
            text=site_measures["sampleID"],
            hoverinfo="text"
        )
        traces.append(trace)
    for trace in traces:
        fig.add_trace(trace, secondary_y=True)    
    fig.update_layout(
        xaxis_title="Date",
        xaxis_tick0="2020-12-27",
        xaxis_dtick=7 * 24 * 3600000,
        xaxis_tickformat="%d-%m-%Y",
        xaxis_tickangle=30, plot_bgcolor="white",
        xaxis_gridcolor="rgba(100,100,100,0.10)",
        yaxis_gridcolor="rgba(0,0,0,0)",
        xaxis_ticks="outside"
    )
    fig.update_yaxes(title="SARS-CoV-2 / PMMoV", secondary_y=True, range=[0, max_normalized])
    if region_name == "Capitale-Nationale":
        region_name = "Région de Québec"
    fig.update_layout(title=dict(text=f"Surveillance SRAS-CoV-2 via les eaux usées<br>{region_name}"))
    return fig

def get_cases_from_ledevoir(region_name, start_date=None, end_date=None):
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    response = requests.get("https://ledevoir-coronavirus.herokuapp.com/api/v2/reports/ca/qc")
    j = json.loads(response.text)
    region_codes = {}
    for i in range(len(j["regions"])):
        region_codes[j["regions"][i]["name"]] = i
    region_code = region_codes[region_name]
    cases = pd.DataFrame(j["regions"][region_code]["data"])
    cases["date"] = pd.to_datetime(cases["date"])
    cases = cases.set_index("date")
    if pd.isna(start_date) and pd.isna(end_date):
        pass
    elif pd.isna(end_date):
        cases = cases.loc[start_date:]
    elif pd.isna(start_date):
        cases = cases.loc[:end_date]
    else:
        cases = cases.loc[start_date : end_date]
    return cases

def create_graph(data, region, sites, max_norm=None):
    graph = graph_normalized(existing_odm, region, sites, max_norm)
    cases = get_cases_from_ledevoir(region, start_date="2021-01-01")
    cases_trace = go.Bar(x=cases.index, y=cases["dc"], name="Nouveaux cas<br>journaliers", marker=dict(opacity=0.3))
    graph.add_trace(cases_trace, secondary_y=False)
    graph.update_layout(legend=dict(yanchor="top", xanchor="left", orientation="h", y=1.1, x=0))
    graph.update_yaxes(title="Nouveaux cas",side="right", secondary_y=False)
    graph.update_yaxes(side="left", secondary_y=True)
    graph.add_layout_image(
    dict(
        source="https://www.centreau.ulaval.ca/fileadmin/Documents/Image_de_marque/102378_MODIF_LOGO-CENTREAU_noir.jpg",
        xref="paper", yref="paper",
        x=1, y=1.00,
        sizex=0.25, sizey=0.25,
        xanchor="right", yanchor="bottom"
    )
)
    graph.show()
    return graph

### Workaround for Kaleido (static image export) bug

In [8]:
region="Capitale-Nationale"
sites = ["qc_1", "qc_2"]
max_norm = 0.2
outlier_thresh = 0.3
qc_graph = create_graph(existing_odm, region, sites, 0.2)

images_folder = "/Users/jeandavidt/Downloads/plotly_images/"
qc_graph.write_image(images_folder+"qc_April1.png", width=1000, scale=3)
qc_graph.write_image(images_folder+"qc_April1.svg", width=1000)




In [9]:
region="Montréal"
sites = ["mtl_11", "mtl_05", "mtl_16", "mtl_12"]
max_norm = 1
graph = create_graph(existing_odm, region, sites, max_norm)

images_folder = "/Users/jeandavidt/Downloads/plotly_images/"
graph.write_image(images_folder+"mtl_April1.png", width=1000, scale=3)
graph.write_image(images_folder+"mtl_April1.svg", width=1000)